In [ ]:
#Import modules
import re
import os
import time
import pandas as pd
import numpy as np
import requests

## Step 1: Get longs and lats

In [ ]:
#Load CSV
df_qv = pd.read_csv('VacateOrderComplaintRecords_Quarantine.csv')

In [ ]:
#Set google geocoding variables
api_key = os.environ.get('google_geo_api')
addresses = df_qv[['Street', 'Borough']].apply(lambda x: ', '.join(x), axis=1)

all_results = []

#Ping Google geocoding API
for address in addresses:
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json'
        keys = {'address': address, 'key': api_key}
        r = requests.get(url,params=keys)
        result_dic = r.json()
        lat = result_dic['results'][0]['geometry']['location']['lat']
        lng = result_dic['results'][0]['geometry']['location']['lng']
        result_dict = {
            'address': address,
            'latitude': lat,
            'longitude': lng
            'type': 'Point'
        }
        all_results.append(result_dict)
        print(result_dict)
    except:
        print(error_count, "ERROR")
    time.sleep(1)

In [ ]:
#Save as dataframe
df_geo = pd.DataFrame(all_results)

In [ ]:
#Save as CSV
df_geo.to_csv('GeoInfo.csv', index=False)

## Step 2: Do properties.stuff

In [ ]:
#Load CSV
df_qv = pd.read_csv('VacateOrderComplaintRecords_Quarantine.csv')
df_qv.head(1)

#Set new df
df_final = df_qv

### Step 2.5: Some mopping up of dodgy values

In [ ]:
#Get rid of complaint null values
df_final['Complaint'] = df_final['Complaint'].fillna("Not on record")
df_final.head(1)

In [ ]:
#Convert ZIPs to string
df_final['ZIP'] = df_final['ZIP'].astype(str)
df_final['properties.headline'] = df_final[['Street', 'Borough', 'ZIP']].apply(lambda x: ', '.join(x), axis=1)
df_final.head(1)

### Back to the actual properties stuff

In [ ]:
#Set properties.article
def article(street, borough, ZIP, owner, vacate_type, date_issued, category_code, comments, complaint, BIN, BIN_URL):
    #Define all the categories in the article, add in HTML
    address = "<h2>" + street + ", " + borough + ", " + ZIP + r"</h2>"
    complaint = "<p><strong>Initial complaint:</strong> " + complaint + "</p>"
    owner = "<p><strong>Owner:</strong> " + owner + "</p>"
    vacate_type = "<p><strong>Vacate type:</strong> " + vacate_type + "</p>"
    date = "<p><strong>Date issued:</strong> " + date_issued + "</p>"
    category = "<p><strong>Category code:</strong> " + category_code + "</p>"
    comments = "<p><strong>Inspection comments:</strong> " + comments + "</p>"
    building_record = '<p><strong>Building record:</strong> <a target="_blank" href="' + BIN_URL + '">' + BIN.astype(str) + "</a></p>"
    #Stick everything together, decide order of appearance
    article = "<div id='article'>" + address + complaint + owner + vacate_type + date + category + comments + building_record + "</div>"
    return article

df_final['properties.article'] = article(df_final['Street'], df_final['Borough'], df_final['ZIP'], df_final['Owner'], df_final['Vacate_Type'], df_final['Date_Issued'], df_final['Category_Code'], df_final['Comments'], df_final['Complaint'], df_final['BIN'], df_final['BIN_URL'])
df_final

In [ ]:
#Set properties.name
df_final['properties.name'] = df_final['properties.headline']
df_final.head(1)

In [ ]:
#Set properties.color
def color(cell):
    if cell == 'Full Vacate':
        hex_color = '#FF2700'
    if cell != 'Full Vacate':
        hex_color = '#FFA700'
    return hex_color

df_final['properties.color'] = df_final['Vacate_Type'].apply(color)
df_final.head(1)

In [ ]:
#Set properties.radius
df_final['properties.radius'] = 5
df_final.head(1)

In [ ]:
#Drop unnecessary columns
df_final = df_final.drop(['Street', 'Borough', 'ZIP', 'BIN', 'BIN_URL', 'Category_Code', 'Owner', 'Date_Issued', 'Vacate_Type', 'Comments', 'Complaint'], axis=1)
df_final.head(1)

## Step 3: Join geo and properties tables

In [ ]:
#Load URLs from CSV
df_geo = pd.read_csv('GeoInfo.csv')

In [ ]:
#Stick coordinates into one column
df_final['geometry.coordinates'] = df_final[['longitude', 'latitude']].values.tolist()
df_final.head(1)

In [ ]:
#Change type to geometry.type
df_final['geometry.type'] = df_final['type']
df_final.head(1)

In [ ]:
#Drop unnecessary columns
df_final = df_final.drop(['latitude', 'longitude', 'type', 'address'], axis=1)
df_final.head(1)

In [ ]:
#Save as CSV just in case
df_final.to_csv('MergedDataframe.csv', index=False)

## Step 4: Export to geojson

In [ ]:
import json

In [ ]:
ok_json = json.loads(df_final.to_json(orient='records'))
ok_json

In [ ]:
#Change format to GeoJSON
def process_to_geojson(file):
    geo_data = {"type": "FeatureCollection", "features":[]}
    for row in file:
        this_dict = {"type": "Feature", "properties":{}, "geometry": {}}
        for key, value in row.items():
            key_names = key.split('.')
            if key_names[0] == 'geometry':
                this_dict['geometry'][key_names[1]] = value
            if str(key_names[0]) == 'properties':
                this_dict['properties'][key_names[1]] = value
        geo_data['features'].append(this_dict)
    return geo_data

In [ ]:
#Run function
geo_format = process_to_geojson(ok_json)
geo_format

In [ ]:
#Output GeoJSON file
with open('geo-data12-3.js', 'w') as outfile:
    outfile.write("infoData = ")
#geojson output
with open('geo-data12-3.js', 'a') as outfile:
    json.dump(geo_format, outfile)